<a href="https://colab.research.google.com/github/lhajabi/PPI_Grad/blob/main/Dataset_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generator

In [ ]:
"""ONLY RUNS IF GPU AVAILABLE
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

gpu()"""

"ONLY RUNS IF GPU AVAILABLE\nimport tensorflow as tf\ndevice_name = tf.test.gpu_device_name()\nif device_name != '/device:GPU:0':\n  raise SystemError('GPU device not found')\nprint('Found GPU at: {}'.format(device_name))\n\ndef gpu():\n  with tf.device('/device:GPU:0'):\n    random_image_gpu = tf.random.normal((100, 100, 100, 3))\n    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)\n    return tf.math.reduce_sum(net_gpu)\n\ngpu()"

In [ ]:
import pandas as pd
import csv
import random
import tensorflow as tf
import numpy as np

from google.colab import drive
"""drive.flush_and_unmount()
!mkdir /content/MyDrive"""
drive.mount('/content/MyDrive')



Mounted at /content/MyDrive


In [ ]:
"""---------------------------------------------------------------------------------------------------------------------
Specify Dataset parameters for the required dataset:
CD_POS: True or False
CD_NEG: True or False
CD_Val: int = 40 or int = 60 (refers to sequences similarity "strictness" ex:60 less strict than 40)
num_samples: int
ratio: float 0 to 1 ratio of negative to posotive samples (ex: 0.9 => 90% neg and 10% pos)
confidence: 'medium' = 0.4 conf or 'high' = 0.7 conf for the positive samples
***last code block in Generator has the save dataset cmd make sure to recomment it so as not to overright existing files
------------------------------------------------------------------------------------------------------------------------"""
CD_POS = True
CD_NEG = False
CD__POS_Val = 40
CD__NEG_Val = 40
num_samples = 200000
ratio = 0.5
confidence = 'high'
seed = 42

POSITIVE PART:

In [ ]:
if CD_POS == False:
  if confidence == 'medium':
    positive_df = pd.read_csv('/content/MyDrive/MyDrive/Data/ONLY POSITIVE/ALL_POS(0.4).txt', delimiter=',')### 1,858,944 samples
  else:
    positive_df = pd.read_csv('/content/MyDrive/MyDrive/Data/ONLY POSITIVE/ALL_POS(0.7).txt', delimiter=',')### 473,860 samples
elif CD__POS_Val !=60:
  if confidence == 'medium':
    positive_df = pd.read_csv('/content/MyDrive/MyDrive/Data/ONLY POSITIVE/ALL_POS_CD40(0.4).txt', delimiter=',')## 421,792 samples
  else:
    positive_df = pd.read_csv('/content/MyDrive/MyDrive/Data/ONLY POSITIVE/ALL_POS_CD40(0.7).txt', delimiter=',')### 122,242 samples
else:
  if confidence == 'medium':
    positive_df = pd.read_csv('/content/MyDrive/MyDrive/Data/ONLY POSITIVE/ALL_POS_CD60(0.4).txt', delimiter=',')### 604,994 samples
  else:
    positive_df = pd.read_csv('/content/MyDrive/MyDrive/Data/ONLY POSITIVE/ALL_POS_CD60(0.7).txt', delimiter=',')### 162,506 samples

NEGATIVE PART:

In [ ]:
if CD_NEG == False:
  negative_df = pd.read_csv('/content/MyDrive/MyDrive/Data/ONLY NEGATIVE/ALL_NEG.txt', delimiter=',')### 196,569 samples
elif CD__NEG_Val !=60:
  negative_df = pd.read_csv('/content/MyDrive/MyDrive/Data/ONLY NEGATIVE/ALL_NEG_CD40.txt', delimiter=',')### 8,176 samples
else:
  negative_df = pd.read_csv('/content/MyDrive/MyDrive/Data/ONLY NEGATIVE/ALL_NEG_CD60.txt', delimiter=',')### 14,259 samples

COMBINE:

In [ ]:
def _scale_down_dataframe(input_df, combined_score, total_samples=10000, random_seed=41):
    if combined_score == 1:
        group_ones = input_df[input_df['combined_score'] == 1]
        sample_ones = group_ones.sample(n=min(total_samples, len(group_ones)), random_state=random_seed)
        return sample_ones
    else:
        group_zeros = input_df[input_df['combined_score'] == 0]
        sample_zeros = group_zeros.sample(n=min(total_samples, len(group_zeros)), random_state=random_seed)
        return sample_zeros

In [ ]:
pos_samples = int(num_samples - (num_samples * ratio))
neg_samples = int(num_samples * ratio)

pos_samples = min(pos_samples, len(positive_df))
neg_samples = min(neg_samples, len(negative_df))

if pos_samples + neg_samples != num_samples:
  if min(pos_samples,neg_samples) ==  neg_samples:
    new_num_samples = neg_samples / ratio
    pos_samples = int(new_num_samples - (new_num_samples * ratio))
  else:
    new_num_samples = pos_samples / ratio
    neg_samples = int(num_samples * ratio)

positive_df_sample = _scale_down_dataframe(input_df = positive_df, combined_score= 1, total_samples= pos_samples, random_seed=seed)
negative_df_sample = _scale_down_dataframe(input_df = negative_df, combined_score= 0, total_samples= neg_samples, random_seed=seed)

final_df = pd.concat([positive_df_sample, negative_df_sample])

final_df = final_df.reset_index(drop=True)
print(final_df['combined_score'].value_counts())
final_df.info()

1    100000
0    100000
Name: combined_score, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   protein1        200000 non-null  object
 1   protein2        200000 non-null  object
 2   combined_score  200000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.6+ MB


In [ ]:
import os
drive_path = '/content/MyDrive/MyDrive/Data/GENERATED DATASETS'

if not os.path.exists(drive_path):
    os.makedirs(drive_path)

os.chdir(drive_path)
#final_df.to_csv('Generated200k_(POS40-0.7)_50:50.txt', index=False) #save your df to txt file


#Tester:

In [ ]:
fasta_file_path = '/content/drive/MyDrive/Data/9606.protein.sequences.v12.0.fa'
